In [ ]:
# install packages for colab/workbench
!pip install --upgrade pip -q
!pip install torch torchvision --upgrade -q
!pip install sentence-transformers==2.2.2 -q
!pip install scikit-learn -q
!pip install transformers==4.21.0 -q

# download the dataset if needed
import os
if not os.path.exists('course_data.csv'):
    !wget -O course_data.csv "https://raw.githubusercontent.com/Bluedata-Consulting/GAAPB01-training-code-base/refs/heads/main/Assignments/assignment2dataset.csv"
    print("Downloaded course dataset")

# Course Recommender System

Building a simple recommendation engine that suggests courses based on what someone's already taken and what they're interested in.

The basic idea: 
- Load course catalog
- Turn course descriptions into vectors using sentence embeddings
- When someone asks for recommendations, find similar courses using cosine similarity
- Return the top 5 matches

In [ ]:
# grab all the packages we need
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import os
from typing import List, Tuple
import warnings
warnings.filterwarnings('ignore')

# try to import sentence transformers, fallback to TF-IDF if it fails
try:
    from sentence_transformers import SentenceTransformer
    USE_SENTENCE_TRANSFORMER = True
    print("Using SentenceTransformer for embeddings")
except ImportError as e:
    print(f"SentenceTransformer not available: {e}")
    print("Falling back to TF-IDF vectorization")
    USE_SENTENCE_TRANSFORMER = False

In [ ]:
# load up the course data
df = pd.read_csv('course_data.csv')

print(f"Got {len(df)} courses to work with")
print(f"Columns: {list(df.columns)}")
print("\nFirst few courses:")
df.head()

In [ ]:
# check for any missing data
print("Missing values:")
print(df.isnull().sum())

print("\nSample course description:")
print(f"Title: {df.iloc[0]['title']}")
print(f"Description: {df.iloc[0]['description'][:200]}...")

# combine title and description for better embeddings
df['full_text'] = df['title'] + '. ' + df['description']
print(f"\nCombined text length avg: {df['full_text'].str.len().mean():.0f} chars")

In [ ]:
# set up the embedding model
embeddings_file = 'course_embeddings.pkl'

if USE_SENTENCE_TRANSFORMER:
    # using sentence transformer (better results)
    model = SentenceTransformer('all-MiniLM-L6-v2')
    print("Loading SentenceTransformer model...")
    
    if os.path.exists(embeddings_file):
        print("Loading existing embeddings...")
        with open(embeddings_file, 'rb') as f:
            course_embeddings = pickle.load(f)
    else:
        print("Creating embeddings with SentenceTransformer (this might take a minute)...")
        course_embeddings = model.encode(df['full_text'].tolist())
        
        with open(embeddings_file, 'wb') as f:
            pickle.dump(course_embeddings, f)
        print("Embeddings saved!")
        
else:
    # fallback to TF-IDF (simpler but still effective)
    print("Using TF-IDF vectorization...")
    model = TfidfVectorizer(max_features=1000, stop_words='english', ngram_range=(1, 2))
    
    if os.path.exists(embeddings_file):
        print("Loading existing TF-IDF embeddings...")
        with open(embeddings_file, 'rb') as f:
            data = pickle.load(f)
            course_embeddings = data['embeddings']
            model = data['model']
    else:
        print("Creating TF-IDF embeddings...")
        course_embeddings = model.fit_transform(df['full_text'].tolist()).toarray()
        
        with open(embeddings_file, 'wb') as f:
            pickle.dump({'embeddings': course_embeddings, 'model': model}, f)
        print("TF-IDF embeddings saved!")

print(f"Embedding shape: {course_embeddings.shape}")
print(f"Using {'SentenceTransformer' if USE_SENTENCE_TRANSFORMER else 'TF-IDF'} for embeddings")

In [ ]:
# the main recommendation function
def recommend_courses(profile: str, completed_ids: List[str] = None) -> List[Tuple[str, float]]:
    """
    Find courses similar to the user profile
    
    profile: what the user is interested in / has done
    completed_ids: course IDs they've already taken (we'll filter these out)
    
    returns: list of (course_id, similarity_score) tuples
    """
    if completed_ids is None:
        completed_ids = []
    
    # turn the user profile into an embedding
    if USE_SENTENCE_TRANSFORMER:
        profile_embedding = model.encode([profile])
    else:
        # use TF-IDF transform
        profile_embedding = model.transform([profile]).toarray()
    
    # find similarities with all courses
    similarities = cosine_similarity(profile_embedding, course_embeddings)[0]
    
    # get course indices sorted by similarity
    similar_indices = np.argsort(similarities)[::-1]
    
    recommendations = []
    for idx in similar_indices:
        course_id = df.iloc[idx]['course_id']
        score = similarities[idx]
        
        # skip if they've already taken it
        if course_id not in completed_ids:
            recommendations.append((course_id, score))
        
        # stop when we have 5 recommendations
        if len(recommendations) >= 5:
            break
    
    return recommendations

# helper function to show course details
def show_recommendations(profile, completed_ids=None):
    print(f"User Profile: {profile}")
    if completed_ids:
        completed_titles = [df[df['course_id'] == cid]['title'].iloc[0] for cid in completed_ids if cid in df['course_id'].values]
        print(f"Completed Courses: {', '.join(completed_titles)}")
    print(f"\nRecommendations (using {'SentenceTransformer' if USE_SENTENCE_TRANSFORMER else 'TF-IDF'}):")
    print("-" * 50)
    
    recs = recommend_courses(profile, completed_ids)
    for i, (course_id, score) in enumerate(recs, 1):
        course_info = df[df['course_id'] == course_id].iloc[0]
        print(f"{i}. {course_info['title']} (ID: {course_id})")
        print(f"   Similarity: {score:.3f}")
        print(f"   Description: {course_info['description'][:150]}...")
        print()

## Testing with the sample queries

Let's try the 5 test cases from the assignment and see what gets recommended.

In [ ]:
# Test 1: Data visualization after Python
print("=" * 60)
print("TEST 1: Data Science Path")
print("=" * 60)
show_recommendations(
    "I've completed the Python Programming for Data Science course and enjoy data visualization. What should I take next?",
    completed_ids=['C016']  # Python Programming for Data Science
)

In [ ]:
# Test 2: Azure containers and CI/CD
print("=" * 60)
print("TEST 2: DevOps/Cloud Path")
print("=" * 60)
show_recommendations(
    "I know Azure basics and want to manage containers and build CI/CD pipelines. Recommend courses.",
    completed_ids=['C007']  # Cloud Computing with Azure
)

In [ ]:
# Test 3: ML to neural networks
print("=" * 60)
print("TEST 3: Advanced ML Path")
print("=" * 60)
show_recommendations(
    "My background is in ML fundamentals; I'd like to specialize in neural networks and production workflows.",
    completed_ids=['C001']  # Foundations of Machine Learning
)

In [ ]:
# Test 4: Microservices with Kubernetes
print("=" * 60)
print("TEST 4: Microservices Path")
print("=" * 60)
show_recommendations(
    "I want to learn to build and deploy microservices with Kubernetes—what courses fit best?"
)

In [ ]:
# Test 5: Blockchain for beginners
print("=" * 60)
print("TEST 5: Blockchain Beginner Path")
print("=" * 60)
show_recommendations(
    "I'm interested in blockchain and smart contracts but have no prior experience. Which courses do you suggest?"
)

## Results Analysis

Let me evaluate how well the recommendations worked for each test case:

**Test 1 - Data Visualization Path:** 
- Should get courses related to data viz, analytics, or advanced Python data tools
- Expected: Tableau, Power BI, data analysis courses
- Assessment: TBD after running

**Test 2 - DevOps/Cloud Path:**
- Looking for container management and CI/CD after Azure basics
- Expected: Docker/Kubernetes, DevOps practices, maybe more cloud stuff
- Assessment: TBD after running

**Test 3 - Advanced ML Path:**
- Wants neural networks and production workflows after ML fundamentals  
- Expected: Deep Learning, MLOps, maybe Computer Vision or NLP
- Assessment: TBD after running

**Test 4 - Microservices Path:**
- Specifically wants Kubernetes and microservices
- Expected: Containerization, APIs/Microservices, DevOps
- Assessment: TBD after running

**Test 5 - Blockchain Beginner:**
- Complete beginner wanting blockchain and smart contracts
- Expected: Blockchain fundamentals course, maybe some programming basics
- Assessment: TBD after running

The similarity scores should help us understand how confident the model is about each recommendation.

In [ ]:
# quick check - let's see what the course catalog looks like
print("Available courses:")
for _, row in df.iterrows():
    print(f"- {row['course_id']}: {row['title']}")